In [1]:
from langchain_openai import AzureChatOpenAI
from langchain.prompts import ChatPromptTemplate
import os

import truststore
truststore.inject_into_ssl()
import numpy as np
import pandas as pd
from tqdm import tqdm
import time
import seaborn as sns
import numpy as np
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt
os.environ["APP_CLIENT_ID"] = "zizhang-chen-research-app"
os.environ["APP_CLIENT_SECRET"] = "cQPcueFP7tDrimbf8NW2GAHcHeQa"
from llm_idam_token_generator.idam_token_generator import get_idam_token
from langchain_community.callbacks import get_openai_callback
from langchain_openai import OpenAI
import tiktoken

In [2]:
end_point_list = [
    'https://lmaas-beta.ai.gehealthcare.com',
    "https://openai-llm-frontdoor-hma7evbthrd4cugn.a01.azurefd.net"
]

model_list = ["gpt-35-turbo-16k", "gpt-4-32k-beta", "gpt-4o"]

# APIM_KEY = "8b96051ed6b84e4dad762fdc9f8c809e"

OPENAI_ENDPOINT = end_point_list[0]
OPENAI_DEPLOYMENT_MODEL = model_list[1]
OPENAI_AZURE_API_VERSION = "2023-12-01-preview"
# OPENAI_AZURE_API_VERSION = '2024-02-01'
OPENAI_TYPE="azure"

In [3]:
llm = AzureChatOpenAI(
    api_key=
    "xxx",  # This is not playing any role, but required as per OpenAI sdk. So any random could be passed.
    azure_endpoint=OPENAI_ENDPOINT,
    deployment_name=OPENAI_DEPLOYMENT_MODEL,
    openai_api_version=OPENAI_AZURE_API_VERSION,
    n=2,
    default_headers={
        'Authorization': f'Bearer {get_idam_token()}',
        'Content-Type': 'application/json'
    })
# prompt = ChatPromptTemplate.from_template("what is the city {person} is from?")
# chain = prompt | llm
# print(chain.invoke({"person": "Narendra modi"}))

PID:3785 INFO llm_idam_token_generator.idam_token_generator - Client ID: zizhang-chen-research-app - Generating new token.
PID:3785 INFO llm_idam_token_generator.idam_token_generator - Client ID: zizhang-chen-research-app - All required environment variables are present.
/opt/anaconda3/lib/python3.11/site-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host '127.0.0.1'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
PID:3785 INFO llm_idam_token_generator.idam_token_generator - Client ID: zizhang-chen-research-app - IDAM Access Token is generated
/opt/anaconda3/lib/python3.11/site-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host '127.0.0.1'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warn

In [4]:
df = pd.read_csv('data/labtest_patient_description.csv')
description_list = df['description'].values
lab_test_list = [i.split('_')[1] for i in df.columns[2:7]]

lab_test_prediction = [
    'Predict whether an anemia lab test result comes back as: normal (>=120 g/L), mild (>=110 and <120 g/L), moderate (>=70 and <110 g/L), or severe (<70 g/L).',
    'Predict whether a hypoglycemia lab test result comes back as: normal (>=3.9 mmol/L), mild (>=3.5 and <3.9 mmol/L), moderate (>=3 and <3.5 mmol/L), or severe (<3 mmol/L).',
    'Predict whether a hyponatremia lab test result comes back as: normal (>=135 mmol/L), mild (>=130 and <135 mmol/L), moderate (>=125 and <130 mmol/L), or severe (<125 mmol/L).',
    'Predict whether a thrombocytopenia lab test result comes back as: normal (>=150 10^9/L), mild (>=100 and <150 10^9/L), moderate (>=50 and <100 10^9/L), or severe (<50 10^9/L).',
    'Predict whether a hyperkalemia lab test result comes back as: normal (<=5.5 mmol/L), mild (>5.5 and <=6mmol/L), moderate (>6 and <=7 mmol/L), or severe (>7 mmol/L).'
]

In [5]:
def split_string_into_two_parts(text):
    lines = text.split('\n')
    first_part = lines[:2]
    second_part = lines[2:]

    first_part_string = '\n'.join(first_part)
    second_part_string = '\n'.join(second_part)

    return first_part_string, second_part_string


def generate_cot():

    chain_of_thought_prompt = [
        "Begin by considering the patient's age and sex, which can influence the prevalence and risk factors for certain conditions.",
        "Review other recent medical events documented in the EHR that might affect lab test results, such as surgeries, new diagnoses, or changes in treatment plans.",
        "1. Anemia: Consider the patient's dietary intake, any recent surgeries like gastrointestinal surgeries, and ongoing treatments like iron supplementation. Check if there are any improvements in hemoglobin and ferritin levels, especially in the context of the patient’s age and sex. Predict if anemia is likely to resolve or persist.",
        "2. Hyperkalemia: Look at kidney function markers and medication influences, but also consider the patient's age-related kidney changes or recent acute kidney events. Predict the stability of potassium levels.",
        "3. Hypoglycemia: Analyze glucose monitoring data, meal patterns, and insulin administration carefully, taking into account any recent episodes of acute illness or stress that could affect glucose metabolism. Make a prediction about potential hypoglycemic events.",
        "4. Hyponatremia: Evaluate fluid balance issues, taking into account any recent conditions that might cause fluid retention or loss, like heart failure or diuretic adjustments. Predict sodium levels in the upcoming tests.",
        "5. Thrombocytopenia: Assess impacts from recent treatments such as chemotherapy, taking into account other factors like viral infections or autoimmune flare-ups that might exacerbate platelet depletion. Provide a prediction on platelet trends."
    ]

    return '\n'.join(cot)

def combin_prompt_cot(general_prompt, person_info, medical_event, template_events_answer_style):
    template_by_task = '**Task:**\n' + general_prompt + '\n'

    template_info = '**Patient age and demographic information:**\n' + person_info + '\n'

    template_events = '**Medical Events:**\n' + medical_event + '\n'

    return template_by_task + '\n' + '{events}' , template_info + '\n' + template_events + template_events_answer_style

def calculate_row_entropy(df):
    def entropy(row):
        counts = row.value_counts(normalize=True)
        return -np.sum(counts * np.log2(counts + np.finfo(float).eps))

    return df.apply(entropy, axis=1)

def num_tokens_from_string(string, encoding_model = 'gpt-3.5-turbo') -> int:
    encoding = tiktoken.encoding_for_model(encoding_model)
    num_tokens = len(encoding.encode(string))
    return num_tokens


def generate_anwer_requirement():
    template_events_answer_style = "**Answer requirement:**\n"

    answer_requirements = "Please only answer with 'Yes' or 'No'"

    lab_test_prediction = [
        'Predict whether an anemia lab test result comes back as normal:',
        'Predict whether a hypoglycemia lab test result comes back as normal:',
        'Predict whether a hyponatremia lab test result comes back as normal:',
        'Predict whether a thrombocytopenia lab test result comes back as normal:',
        'Predict whether a hyperkalemia lab test result comes back as normal:'
    ]

    modified_list = [
        s + " " + answer_requirements for s in lab_test_prediction
    ]

    final_string_requirements = '\n'.join(modified_list)

    final_string_requirements = template_events_answer_style + final_string_requirements

    example_answer = "\n\n**Answer example:**\n"

    example_prediction = "anemia: Yes\n" + "hypoglycemia: No\n" + "hyponatremia: No\n" + "thrombocytopenia: No\n" + "hyperkalemia: Yes"

    final_string_example = example_answer + example_prediction

    final_string = final_string_requirements + final_string_example

    return final_string


def generate_prompt_task(lab_test_prediction):

    multi_task_prompt = '\n'.join(lab_test_prediction)

    task = f"You are an experienced doctor. Based on the provided patient age, demographic information, medical events, and other relevant lab test results prediction you made at the same time. Use your medical knowledge and reasoning to: \n{multi_task_prompt} right after the given medical events.\nPlease only answer with 'Yes' or 'No'. Yes if the patient lab results come back as normal, No if the patient lab results remain moderate or high. Please respond 'Yes' if it is at all plausible, only use 'No' if absolutely certain otherwise."
    return task

In [6]:
out_csv_names = 'multi_task_lab_test.csv'
general_prompt = generate_prompt_task(lab_test_prediction=lab_test_prediction)
# gt_list = df['value_'+lab_test_list[idx]].values
gt_list = df.loc[:, ['value_' + lab_test_list[i] for i in range(len(lab_test_list))]].values
gt_list = pd.DataFrame(gt_list)
with get_openai_callback() as cb:

    answer_list_all = []

    for i in tqdm(range(len(df))):
        # for i in tqdm(range(50, 100)):
        # for i in tqdm(range(1)):
        row = df.loc[i, :]
        answer_list_5 = []
        for _ in range(5):
            # for _ in range(1):
            current_description = description_list[i]
            person_info, medical_events = split_string_into_two_parts(
                current_description)
            anwer_requirement = generate_anwer_requirement()
            c, e = combin_prompt_cot(general_prompt, person_info,
                                     medical_events, anwer_requirement)
            prompt = ChatPromptTemplate.from_template(c)
            chain = prompt | llm
            result = chain.invoke({'events': e})
            answer_list_5.append(result.content)
        answer_list_all.append(answer_list_5)
    df_pred = pd.DataFrame(answer_list_all)
    df_pred.columns = ['pred_1', 'pred_2', 'pred_3', 'pred_4', 'pred_5']
    df_pred = pd.concat([df_pred, gt_list], axis=1)
    # df_pred.to_csv(os.path.join('results/gpt35_new/cross_task_labtest/',
    #                             out_csv_names),
    #                index=False)
    df_pred.to_csv(os.path.join('results/gpt4_new/cross_task_labtest/', out_csv_names), index=False)

  0%|          | 0/100 [00:00<?, ?it/s]PID:3785 INFO httpx - Client ID: zizhang-chen-research-app - HTTP Request: POST https://lmaas-beta.ai.gehealthcare.com/openai/deployments/gpt-4-32k-beta/chat/completions?api-version=2023-12-01-preview "HTTP/1.1 200 OK"
PID:3785 INFO httpx - Client ID: zizhang-chen-research-app - HTTP Request: POST https://lmaas-beta.ai.gehealthcare.com/openai/deployments/gpt-4-32k-beta/chat/completions?api-version=2023-12-01-preview "HTTP/1.1 200 OK"
PID:3785 INFO httpx - Client ID: zizhang-chen-research-app - HTTP Request: POST https://lmaas-beta.ai.gehealthcare.com/openai/deployments/gpt-4-32k-beta/chat/completions?api-version=2023-12-01-preview "HTTP/1.1 200 OK"
PID:3785 INFO httpx - Client ID: zizhang-chen-research-app - HTTP Request: POST https://lmaas-beta.ai.gehealthcare.com/openai/deployments/gpt-4-32k-beta/chat/completions?api-version=2023-12-01-preview "HTTP/1.1 200 OK"
PID:3785 INFO httpx - Client ID: zizhang-chen-research-app - HTTP Request: POST https

In [11]:
cb

Tokens Used: 2473264
	Prompt Tokens: 2435765
	Completion Tokens: 37499
Successful Requests: 500
Total Cost (USD): $150.6457800000001

In [7]:
# pd.concat([gt_list, gt_list])

In [8]:
print(c)

**Task:**
You are an experienced doctor. Based on the provided patient age, demographic information, medical events, and other relevant lab test results prediction you made at the same time. Use your medical knowledge and reasoning to: 
Predict whether an anemia lab test result comes back as: normal (>=120 g/L), mild (>=110 and <120 g/L), moderate (>=70 and <110 g/L), or severe (<70 g/L).
Predict whether a hypoglycemia lab test result comes back as: normal (>=3.9 mmol/L), mild (>=3.5 and <3.9 mmol/L), moderate (>=3 and <3.5 mmol/L), or severe (<3 mmol/L).
Predict whether a hyponatremia lab test result comes back as: normal (>=135 mmol/L), mild (>=130 and <135 mmol/L), moderate (>=125 and <130 mmol/L), or severe (<125 mmol/L).
Predict whether a thrombocytopenia lab test result comes back as: normal (>=150 10^9/L), mild (>=100 and <150 10^9/L), moderate (>=50 and <100 10^9/L), or severe (<50 10^9/L).
Predict whether a hyperkalemia lab test result comes back as: normal (<=5.5 mmol/L), mil

In [9]:
print(e[-670:])

rded.

**Answer requirement:**
Predict whether an anemia lab test result comes back as normal: Please only answer with 'Yes' or 'No'
Predict whether a hypoglycemia lab test result comes back as normal: Please only answer with 'Yes' or 'No'
Predict whether a hyponatremia lab test result comes back as normal: Please only answer with 'Yes' or 'No'
Predict whether a thrombocytopenia lab test result comes back as normal: Please only answer with 'Yes' or 'No'
Predict whether a hyperkalemia lab test result comes back as normal: Please only answer with 'Yes' or 'No'

**Answer example:**
anemia: Yes
hypoglycemia: No
hyponatremia: No
thrombocytopenia: No
hyperkalemia: Yes


In [10]:
# question_prompt = "\n".join(lab_test_prediction)
# print(question_prompt)

In [13]:
df.columns

Index(['patient_id', 'prediction_time', 'value_anemia', 'value_hypoglycemia',
       'value_hyponatremia', 'value_thrombocytopenia', 'value_hyperkalemia',
       'description'],
      dtype='object')